In [16]:
"""
Returns a Vector of vectors for the movement of every point, given the forces vector of vectors
If no break occurs then there will be only one motion returned
"""
function get_motions_V(forces::Vector{Vector{Float64}}, strength)

    ndims = length(forces[1])
    nforces = length(forces)
    motions = map(similar,forces)    
    
    for dim in 1:ndims
        north_force = 0.0
        south_force = 0.0
        for f_ii in 1:nforces
            force = forces[f_ii][dim]
            if force>0.0
                north_force+=force
            else
                south_force+=force
            end
        end
        tension = 2*min(north_force, south_force) #This is the resisted force. It that can break, rather than move. 
        if tension>strength

            for f_ii in 1:nforces
                force = forces[f_ii]
                if force>0.0
                    motion[f_ii][dim] = (north_force-strength) 
                else
                    motion[f_ii][dim] = (strength-south_force) 
                end
            end 
        else
            #does not break, so all forces apply same motion
            for f_ii in 1:nforces
                motions[f_ii][dim] = (north_force-strength)                 
            end          
        end
    end
    #Find unique motion rows.
    #These corespond to all the new points
    unique(motions)
end

get_motions_V

In [2]:
"""
Returns a column for the movement of every point, given the forces matrix.
If no break occurs then there will be only one motion returned
"""
function get_motions(forces::Matrix, strength)
    motion = similar(forces)
    
    for dim in 1:size(forces,1)
        is_north = forces[dim,:].>0
        north_force = sum(forces[dim,is_north])
        south_force = -sum(forces[dim,!is_north])
        @assert(north_force>=0)
        @assert(south_force>=0)
        
        tension = 2*min(north_force, south_force) #This is the resisted force. It that can break, rather than move. 
        if tension>strength
            #breaks, so north forces will go one way, and south the other
            motion[dim,is_north]=(north_force-strength) #/sum(is_north)
            motion[dim,!is_north]=(strength-south_force)  #/sum(!is_north)
        else
            #does not break, so all forces apply same motion
            motion[dim,:]=(north_force-south_force)     #/size(forces,1)
        end
    end
    #Find unique motion rows.
    #These corespond to all the new points
    unique(motion,2)
end

get_motions

In [26]:
"""
Returns a column for the movement of every point, given the forces matrix.
If no break occurs then there will be only one motion returned
"""
function get_motions_T(forces::Matrix, strength)
    motion = similar(forces)
    
    for dim in 1:size(forces,2)
        is_north = forces[:,dim].>0
        north_force = sum(forces[is_north,dim])
        south_force = -sum(forces[!is_north,dim])
        @assert(north_force>=0)
        @assert(south_force>=0)
        
        tension = 2*min(north_force, south_force) #This is the resisted force. It that can break, rather than move. 
        if tension>strength
            #breaks, so north forces will go one way, and south the other
            motion[is_north, dim]=(north_force-strength) #/sum(is_north)
            motion[!is_north, dim]=(strength-south_force)  #/sum(!is_north)
        else
            #does not break, so all forces apply same motion
            motion[:,dim]=(north_force-south_force)     #/size(forces,1)
        end
    end
    #Find unique motion rows.
    #These corespond to all the new points
    unique(motion,1)
end

get_motions_T

In [30]:
function test_vectors()
    for round in 1:100

        forces = Vector{Float64}[rand(300) for ii in 1:500]
        get_motions_V(forces, 0.4)
    end
end

test_vectors (generic function with 1 method)

In [31]:
function test_matrixes()
    for round in 1:100
        forces  = rand(300,1)
        for ii in 1:500-1
            forces = [forces rand(300,1)]
        end
        get_motions(forces, 0.4)
    end
end

test_matrixes (generic function with 1 method)

In [32]:
function test_matrixes_T()
    for round in 1:100
        forces  = rand(1,300)
        for ii in 1:500-1
            forces = [forces; rand(1,300)]
        end
        get_motions_T(forces, 0.4)
    end
end

test_matrixes_T (generic function with 1 method)

In [33]:
gc()
@time test_matrixes()

 17.366846 seconds (976.06 k allocations: 28.728 GB, 12.79% gc time)


In [34]:
gc()
@time test_vectors()

  1.697552 seconds (113.49 k allocations: 245.501 MB, 2.40% gc time)


In [35]:
gc()
@time test_matrixes_T()

 17.362118 seconds (1.13 M allocations: 28.734 GB, 12.68% gc time)


In [ ]:
function move!(wordsense_embeddings::Matrix,
    wordsense_col::Int,
               motions::Matrix)
    cur_pos=wordsense_embeddings[:,wordsense_col]
    @show motions
    @show cur_pos
    new_points = motions .+ cur_pos
    #PREM-OPT: The code below works for any number of new points. But if there is only one new point (i.e. no split occured), then we can just replacer the current single point
    other_senses = wordsense_embeddings[:,[1:wordsense_col-1;wordsense_col+1:end]]
    wordsense_embeddings = [other_senses new_points]
end

In [ ]:
word_embeddings = Dict{AbstractString,Matrix}()
word_embeddings["Hi"]=[1. 0; 0 1; 2 3]' #One column per word sense
fs = [1. 2; -1 -2; -0.5 -1; -0.5 0]'
ms=get_motions(fs,2)
move!(word_embeddings["Hi"],2,ms)